In [427]:
import os
import csv
import xlrd
from zipfile import ZipFile
import pprint
import json
import codecs
import requests
import numpy as np

## 1. Quiz: Using CSV Module

In [161]:
#####################################
#                 1                 #
#####################################
"""
Your task is to process the supplied file and use the csv module to extract data from it.
The data comes from NREL (National Renewable Energy Laboratory) website. Each file
contains information from one meteorological station, in particular - about amount of
solar and wind energy for each hour of day.

Note that the first line of the datafile is neither data entry, nor header. It is a line
describing the data source. You should extract the name of the station from it.

The data should be returned as a list of lists (not dictionaries).
You can use the csv modules "reader" method to get data in such format.
Another useful method is next() - to get the next line from the iterator.
You should only change the parse_file function.
"""

'\nYour task is to process the supplied file and use the csv module to extract data from it.\nThe data comes from NREL (National Renewable Energy Laboratory) website. Each file\ncontains information from one meteorological station, in particular - about amount of\nsolar and wind energy for each hour of day.\n\nNote that the first line of the datafile is neither data entry, nor header. It is a line\ndescribing the data source. You should extract the name of the station from it.\n\nThe data should be returned as a list of lists (not dictionaries).\nYou can use the csv modules "reader" method to get data in such format.\nAnother useful method is next() - to get the next line from the iterator.\nYou should only change the parse_file function.\n'

In [162]:
DATADIR = "P4L2_Resources/"
DATAFILE = "745090.csv"

In [163]:
def parse_file(datafile):
    name = ""
    data = []
    with open(datafile,'rb') as f:
        reader = csv.reader(f)
        name = reader.next()[1]
        reader.next()
        for row in reader:
                
            data.append(row)
            
    # Do not change the line below
    return (name, data)

In [164]:
# test if the code above is correct

def test():
    datafile = os.path.join(DATADIR, DATAFILE)
    name, data = parse_file(datafile)

    assert name == "MOUNTAIN VIEW MOFFETT FLD NAS"
    assert data[0][1] == "01:00"
    assert data[2][0] == "01/01/2005"
    assert data[2][5] == "2"
    

if __name__ == "__main__":
    test()

In [165]:
test()

## 2. Quiz: Excel To CSV

In [166]:
#####################################
#                 2                 #
#####################################
# -*- coding: utf-8 -*-
'''
Find the time and value of max load for each of the regions
COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST
and write the result out in a csv file, using pipe character | as the delimiter.

An example output can be seen in the "example.csv" file.
'''

'\nFind the time and value of max load for each of the regions\nCOAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST\nand write the result out in a csv file, using pipe character | as the delimiter.\n\nAn example output can be seen in the "example.csv" file.\n'

In [167]:
# expected output
# Station|Year|Month|Day|Hour|Max Load
# COAST|2013|01|01|10|12345.6
# EAST|2013|01|01|10|12345.6
# FAR_WEST|2013|01|01|10|12345.6
# NORTH|2013|01|01|10|12345.6
# NORTH_C|2013|01|01|10|12345.6
# SOUTHERN|2013|01|01|10|12345.6
# SOUTH_C|2013|01|01|10|12345.6
# WEST|2013|01|01|10|12345.6

In [303]:
datafile = "P4L1_Resources/2013_ERCOT_Hourly_Load_Data.xls"
outfile = "P4L2_Resources/2013_Max_Loads.csv"


In [304]:
# extract csv file from zip file
def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()
        

In [416]:

def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    data = None
# YOUR CODE HERE
# Remember that you can use xlrd.xldate_as_tuple(sometime, 0) to convert
# Excel date to Python tuple of (year, month, day, hour, minute, second)
    data_for_newFrame = []
    # read data on sheet
    data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]
    
    # read each column in data and look for max value
    # and its date and load amount
    for q in range(1,9):
        cv = sheet.col_values(q, start_rowx=1, end_rowx=None)
        station = sheet.cell_value(0,q)
        maxval = max(cv)
        
        maxpos = cv.index(maxval) + 1
        maxtime = sheet.cell_value(maxpos, 0)

        realtime = xlrd.xldate_as_tuple(maxtime, 0)
        print(realtime)

        data_for_newFrame.append([str(station), realtime, maxval])
#         data[station] = {
#                        "maxval": maxval,
# #                        "maxtime": realtime,
#                          }
#     #print data
    return data_for_newFrame


In [417]:
parse_file(datafile)

(2013, 8, 13, 17, 0, 0)
(2013, 8, 5, 17, 0, 0)
(2013, 6, 26, 17, 0, 0)
(2013, 8, 7, 17, 0, 0)
(2013, 8, 7, 18, 0, 0)
(2013, 8, 8, 16, 0, 0)
(2013, 8, 8, 18, 0, 0)
(2013, 8, 7, 17, 0, 0)


[['COAST', (2013, 8, 13, 17, 0, 0), 18779.025510000003],
 ['EAST', (2013, 8, 5, 17, 0, 0), 2380.1654089999956],
 ['FAR_WEST', (2013, 6, 26, 17, 0, 0), 2281.2722140000024],
 ['NORTH', (2013, 8, 7, 17, 0, 0), 1544.7707140000005],
 ['NORTH_C', (2013, 8, 7, 18, 0, 0), 24415.570226999993],
 ['SOUTHERN', (2013, 8, 8, 16, 0, 0), 5494.157645],
 ['SOUTH_C', (2013, 8, 8, 18, 0, 0), 11433.30491600001],
 ['WEST', (2013, 8, 7, 17, 0, 0), 1862.6137649999998]]

In [418]:
def save_file(data, filename):
    # YOUR CODE HERE
    with open(filename, "w") as f:
        w = csv.writer(f, delimiter="|")
        w.writerow(["Station","Year","Month","Day","Hour","Max Load"])
        for s in data:
            year, month, day, hour, _ ,_ = s[1]
            print(s[0], year, month, day, hour, s[2])
            w.writerow([s[0], year, month, day, hour, s[2]])     
# #             year, month, day, hour, _ ,_ = data[s]["maxtime"]
#             w.writerow([s, year, month, day, hour, data[s]["maxval"]])

In [419]:
save_file(parse_file(datafile), outfile)

(2013, 8, 13, 17, 0, 0)
(2013, 8, 5, 17, 0, 0)
(2013, 6, 26, 17, 0, 0)
(2013, 8, 7, 17, 0, 0)
(2013, 8, 7, 18, 0, 0)
(2013, 8, 8, 16, 0, 0)
(2013, 8, 8, 18, 0, 0)
(2013, 8, 7, 17, 0, 0)
('COAST', 2013, 8, 13, 17, 18779.025510000003)
('EAST', 2013, 8, 5, 17, 2380.1654089999956)
('FAR_WEST', 2013, 6, 26, 17, 2281.2722140000024)
('NORTH', 2013, 8, 7, 17, 1544.7707140000005)
('NORTH_C', 2013, 8, 7, 18, 24415.570226999993)
('SOUTHERN', 2013, 8, 8, 16, 5494.157645)
('SOUTH_C', 2013, 8, 8, 18, 11433.30491600001)
('WEST', 2013, 8, 7, 17, 1862.6137649999998)


In [420]:
def test():
    open_zip(datafile)
    data = parse_file(datafile)
    save_file(data, outfile)

    number_of_rows = 0
    stations = []

    ans = {'FAR_WEST': {'Max Load': '2281.2722140000024',
                        'Year': '2013',
                        'Month': '6',
                        'Day': '26',
                        'Hour': '17'}}
    correct_stations = ['COAST', 'EAST', 'FAR_WEST', 'NORTH',
                        'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
    fields = ['Year', 'Month', 'Day', 'Hour', 'Max Load']

    with open(outfile) as of:
        csvfile = csv.DictReader(of, delimiter="|")
        for line in csvfile:
            print(line)
            station = line['Station']
            
            if station == 'FAR_WEST':
                for field in fields:
                    #print(field)
                    # Check if 'Max Load' is within .1 of answer
                    if field == 'Max Load':
                        max_answer = round(float(ans[station][field]), 1)
                        max_line = round(float(line[field]), 1)
                        assert max_answer == max_line
    

                    # Otherwise check for equality
                    else:
                        print(ans[station][field])
                        print(line[field])
                        assert ans[station][field] == line[field]

            number_of_rows += 1
            stations.append(station)

        # Output should be 8 lines not including header
        assert number_of_rows == 8

        # Check Station Names
        assert set(stations) == set(correct_stations)

        
if __name__ == "__main__":\
    test()

(2013, 8, 13, 17, 0, 0)
(2013, 8, 5, 17, 0, 0)
(2013, 6, 26, 17, 0, 0)
(2013, 8, 7, 17, 0, 0)
(2013, 8, 7, 18, 0, 0)
(2013, 8, 8, 16, 0, 0)
(2013, 8, 8, 18, 0, 0)
(2013, 8, 7, 17, 0, 0)
('COAST', 2013, 8, 13, 17, 18779.025510000003)
('EAST', 2013, 8, 5, 17, 2380.1654089999956)
('FAR_WEST', 2013, 6, 26, 17, 2281.2722140000024)
('NORTH', 2013, 8, 7, 17, 1544.7707140000005)
('NORTH_C', 2013, 8, 7, 18, 24415.570226999993)
('SOUTHERN', 2013, 8, 8, 16, 5494.157645)
('SOUTH_C', 2013, 8, 8, 18, 11433.30491600001)
('WEST', 2013, 8, 7, 17, 1862.6137649999998)
{'Hour': '17', 'Month': '8', 'Station': 'COAST', 'Year': '2013', 'Day': '13', 'Max Load': '18779.025510000003'}
{'Hour': '17', 'Month': '8', 'Station': 'EAST', 'Year': '2013', 'Day': '5', 'Max Load': '2380.1654089999956'}
{'Hour': '17', 'Month': '6', 'Station': 'FAR_WEST', 'Year': '2013', 'Day': '26', 'Max Load': '2281.2722140000024'}
2013
2013
6
6
26
26
17
17
{'Hour': '17', 'Month': '8', 'Station': 'NORTH', 'Year': '2013', 'Day': '7', 'Max

In [421]:
test()

(2013, 8, 13, 17, 0, 0)
(2013, 8, 5, 17, 0, 0)
(2013, 6, 26, 17, 0, 0)
(2013, 8, 7, 17, 0, 0)
(2013, 8, 7, 18, 0, 0)
(2013, 8, 8, 16, 0, 0)
(2013, 8, 8, 18, 0, 0)
(2013, 8, 7, 17, 0, 0)
('COAST', 2013, 8, 13, 17, 18779.025510000003)
('EAST', 2013, 8, 5, 17, 2380.1654089999956)
('FAR_WEST', 2013, 6, 26, 17, 2281.2722140000024)
('NORTH', 2013, 8, 7, 17, 1544.7707140000005)
('NORTH_C', 2013, 8, 7, 18, 24415.570226999993)
('SOUTHERN', 2013, 8, 8, 16, 5494.157645)
('SOUTH_C', 2013, 8, 8, 18, 11433.30491600001)
('WEST', 2013, 8, 7, 17, 1862.6137649999998)
{'Hour': '17', 'Month': '8', 'Station': 'COAST', 'Year': '2013', 'Day': '13', 'Max Load': '18779.025510000003'}
{'Hour': '17', 'Month': '8', 'Station': 'EAST', 'Year': '2013', 'Day': '5', 'Max Load': '2380.1654089999956'}
{'Hour': '17', 'Month': '6', 'Station': 'FAR_WEST', 'Year': '2013', 'Day': '26', 'Max Load': '2281.2722140000024'}
2013
2013
6
6
26
26
17
17
{'Hour': '17', 'Month': '8', 'Station': 'NORTH', 'Year': '2013', 'Day': '7', 'Max

## 3. Quiz: Wrangling JSON

In [485]:
#####################################
#                 3                 #
#####################################
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
This exercise shows some important concepts that you should be aware about:
- using codecs module to write unicode files
- using authentication with web APIs
- using offset when accessing web APIs

To run this code locally you have to register at the NYTimes developer site 
and get your own API key. You will be able to complete this exercise in our UI
without doing so, as we have provided a sample result. (See the file 
'popular-viewed-1.json' from the tabs above.)


"Here's your API Key for the Article Search API: beb94741d85547e5af1f0c10c72e58ec"


Your task is to modify the article_overview() function to process the saved
file that represents the most popular articles (by view count) from the last
day, and return a tuple of variables containing the following data:
- labels: list of dictionaries, where the keys are the "section" values and
  values are the "title" values for each of the retrieved articles.
- urls: list of URLs for all 'media' entries with "format": "Standard Thumbnail"

All your changes should be in the article_overview() function. See the test() 
function for examples of the elements of the output lists.
The rest of functions are provided for your convenience, if you want to access
the API by yourself.
"""

'\nThis exercise shows some important concepts that you should be aware about:\n- using codecs module to write unicode files\n- using authentication with web APIs\n- using offset when accessing web APIs\n\nTo run this code locally you have to register at the NYTimes developer site \nand get your own API key. You will be able to complete this exercise in our UI\nwithout doing so, as we have provided a sample result. (See the file \n\'popular-viewed-1.json\' from the tabs above.)\n\n\n"Here\'s your API Key for the Article Search API: beb94741d85547e5af1f0c10c72e58ec"\n\n\nYour task is to modify the article_overview() function to process the saved\nfile that represents the most popular articles (by view count) from the last\nday, and return a tuple of variables containing the following data:\n- labels: list of dictionaries, where the keys are the "section" values and\n  values are the "title" values for each of the retrieved articles.\n- urls: list of URLs for all \'media\' entries with "

In [486]:

URL_MAIN = "http://api.nytimes.com/svc/"
URL_POPULAR = URL_MAIN + "mostpopular/v2/"
API_KEY = { "popular": "beb94741d85547e5af1f0c10c72e58ec",
            "article": "beb94741d85547e5af1f0c10c72e58ec"}


In [487]:
def get_from_file(kind, period):
    filename = "popular-{0}-{1}.json".format(kind, period)
    with open(filename, "r") as f:
        return json.loads(f.read())


In [488]:

def article_overview(kind, period):
    data = get_from_file(kind, period)
    titles = []
    urls =[]
    
    # YOUR CODE HERE
    for article in data:
        section = article["section"]
        title = article["title"]
        titles.append({section: title})
        if "media" in article:
            for m in article["media"]:
                for mm in m["media-metadata"]:
                    if mm["format"] == "Standard Thumbnail":
                        urls.append(mm["url"])

    return (titles, urls)


In [489]:

def query_site(url, target, offset):
    # This will set up the query with the API key and offset
    # Web services often use offset paramter to return data in small chunks
    # NYTimes returns 20 articles per request, if you want the next 20
    # You have to provide the offset parameter
    if API_KEY["popular"] == "" or API_KEY["article"] == "":
        print "You need to register for NYTimes Developer account to run this program."
        print "See Intructor notes for information"
        return False
    params = {"api-key": API_KEY[target], "offset": offset}
    r = requests.get(url, params = params)

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


In [490]:


def get_popular(url, kind, days, section="all-sections", offset=0):
    # This function will construct the query according to the requirements of the site
    # and return the data, or print an error message if called incorrectly
    if days not in [1,7,30]:
        print "Time period can be 1,7, 30 days only"
        return False
    if kind not in ["viewed", "shared", "emailed"]:
        print "kind can be only one of viewed/shared/emailed"
        return False

    url += "most{0}/{1}/{2}.json".format(kind, section, days)
    data = query_site(url, "popular", offset)

    return data


In [491]:
def save_file(kind, period):
    # This will process all results, by calling the API repeatedly with supplied offset value,
    # combine the data and then write all results in a file.
    data = get_popular(URL_POPULAR, "viewed", 1)
    num_results = data["num_results"]
    full_data = []
    with codecs.open("popular-{0}-{1}.json".format(kind, period), encoding='utf-8', mode='w') as v:
        for offset in range(0, num_results, 20):        
            data = get_popular(URL_POPULAR, kind, period, offset=offset)
            full_data += data["results"]
        
        v.write(json.dumps(full_data, indent=2))




In [492]:
# test if the code above is correct

def test():
    
    save_file(kind, period)
    
    titles, urls = article_overview("viewed", 1)
    
    assert len(titles) == 20
    assert len(urls) == 30
    assert titles[2] == {'Opinion': 'Professors, We Need You!'}
    assert urls[20] == 'http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-thumbStandard.jpg'


if __name__ == "__main__":
    test()

AssertionError: 

In [ ]:
data